<h3>Векторизуем словарь Meddra</h3>

In [1]:
from vectorization import ConceptVectorizer

In [2]:
#модели нам не нужны, ведь вложения получены в ячейке fit_transform() и сохранены, поэтому use_model = False
#чтобы получить вложения, надо вызвать CV с use_model = True и вызвать fit_transform
CV_train = ConceptVectorizer('DeepPavlov/rubert-base-cased', '../../Data/External/pt_rus.asc', \
                             use_concept_less=False, use_model=False)
CV_test = ConceptVectorizer('DeepPavlov/rubert-base-cased', '../../Data/External/pt_rus.asc', \
                            use_concept_less=True, use_model=False)
CV_test_without_conceptless = ConceptVectorizer('DeepPavlov/rubert-base-cased', '../../Data/External/pt_rus.asc', \
                            use_concept_less=False, use_model=False)

Some weights of the model checkpoint at DeepPavlov/rubert-base-cased were not used when initializing BertModel: ['cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.predictions.decoder.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of the model checkpoint at DeepPavlov/rubert-base-cased were not used when initializing BertModel: 

In [4]:
CV.fit_transform(mode='mean_pooling')

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


getting concept embeddings in mean_pooling mode...
Compute embeddings...
Embedding aggregation...


In [ ]:
import torch

torch.save(CV.thesaurus_embeddings, 'rubert_thesaurus_embeddings.pt')

In [3]:
import torch

CV_train.thesaurus_embeddings = torch.load('rubert_thesaurus_embeddings.pt')
CV_train.normalization_mode = 'mean_pooling'

CV_test.thesaurus_embeddings = torch.load('rubert_thesaurus_embeddings.pt')
CV_test.normalization_mode = 'mean_pooling'

CV_test_without_conceptless.thesaurus_embeddings = torch.load('rubert_thesaurus_embeddings.pt')
CV_test_without_conceptless.normalization_mode = 'mean_pooling'

In [4]:
CV_train.thesaurus_embeddings

tensor([[ 0.7665, -0.2175,  0.5759,  ...,  0.8289,  0.8725,  0.5172],
        [ 0.5923, -0.2154,  0.5740,  ...,  0.7217,  0.2828,  0.8354],
        [ 0.5225, -0.0286,  0.5411,  ...,  0.4551,  0.5698,  0.6626],
        ...,
        [ 0.9542, -0.1843,  1.0362,  ...,  1.0432,  0.4805,  0.5036],
        [ 0.0918,  0.0579,  0.0690,  ..., -0.1807,  1.0918,  0.5340],
        [ 0.8576, -0.2527,  0.6392,  ...,  0.0424,  1.0747,  0.6844]])

In [3]:
ds[0]

NameError: name 'ds' is not defined

<h3>Создадим датасет RDR</h3>

In [7]:
import jsonlines
import numpy as np
from sklearn.model_selection import train_test_split

In [8]:
ds = []
with jsonlines.open('../../Data/Raw/medNorm_16022022.jsonlines') as reader:
    for obj in reader:
        ds.append(obj)

In [9]:
ds[0]

{'meta': {'fileName': '1000239.xmi',
  'annotator': 'CURATION_USER',
  'corpus': 'main_2800'},
 'raw': 'http://otzovik.com/review_1000239.html\nTITLE\nОтзыв: Оксолиновая мазь Vishpha - бесполезная\nTEXT\nПо многочисленным отзывам знакомых решила и я попробовать эту чудо-мазь, но мрои ожидания не оправдались(Болела как раз наша бабушка и мы решили подстраховаться что бы не заболеть, но увы через 3 дня все были с простудой. Так что исходя из собственного опыта советовать ее не могу. Общее впечатление : бесполезная\nRATING\n2\nNOTE\n',
 'sentences': [[{'forma': 'http', 'start': 0, 'end': 4},
   {'forma': ':', 'start': 4, 'end': 5},
   {'forma': '/', 'start': 5, 'end': 6},
   {'forma': '/', 'start': 6, 'end': 7},
   {'forma': 'otzovik.com', 'start': 7, 'end': 18},
   {'forma': '/', 'start': 18, 'end': 19},
   {'forma': 'review', 'start': 19, 'end': 25},
   {'forma': '_', 'start': 25, 'end': 26},
   {'forma': '1000239', 'start': 26, 'end': 33},
   {'forma': '.', 'start': 33, 'end': 34},
   

In [10]:
print('http://otzovik.com/review_1000239.html\nTITLE\nОтзыв: Оксолиновая мазь Vishpha - бесполезная\nTEXT\nПо многочисленным отзывам знакомых решила и я попробовать эту чудо-мазь, но мрои ожидания не оправдались(Болела как раз наша бабушка и мы решили подстраховаться что бы не заболеть, но увы через 3 дня все были с простудой. Так что исходя из собственного опыта советовать ее не могу. Общее впечатление : бесполезная\nRATING\n2\nNOTE\n')

http://otzovik.com/review_1000239.html
TITLE
Отзыв: Оксолиновая мазь Vishpha - бесполезная
TEXT
По многочисленным отзывам знакомых решила и я попробовать эту чудо-мазь, но мрои ожидания не оправдались(Болела как раз наша бабушка и мы решили подстраховаться что бы не заболеть, но увы через 3 дня все были с простудой. Так что исходя из собственного опыта советовать ее не могу. Общее впечатление : бесполезная
RATING
2
NOTE



In [6]:
X_train, X_test = train_test_split(ds, test_size=0.33, random_state=42)

In [7]:
#выцепим фразы с нормализацией по Meddra без их контекста
train_phrases = []
train_sentences = []
train_concepts = []

test_phrases = []
test_sentences = []
test_concepts = []

test_phrases_without_conceptless = []
test_concepts_without_conceptless = []

log_markup_errors = []

#в трейне не будем собирать conceptless термины
USE_CONCEPT_LESS = False
for review in X_train:
    for ent in review['objects']['MedEntity']:
        if 'MedDRA' in ent.keys():
            if ent['MedDRA']=='':
                if USE_CONCEPT_LESS:
                    ent['MedDRA'] = 'CONCEPT_LESS'
                else:
                    continue
            try:
                train_concepts.append(CV_train.meddra_term_to_meddra_code[ent['MedDRA'].split('|')[0]])
                train_phrases.append(ent['text'])
            except KeyError:
                log_markup_errors.append({'review_id': review['meta']['fileName'], 'entity_id': ent['xmiID']})
            

USE_CONCEPT_LESS = True
for review in X_test:
    for ent in review['objects']['MedEntity']:
        if 'MedDRA' in ent.keys():
            if ent['MedDRA']=='':
                if USE_CONCEPT_LESS:
                    ent['MedDRA'] = 'CONCEPT_LESS'
                else:
                    continue
            try:
                test_concepts.append(CV_test.meddra_term_to_meddra_code[ent['MedDRA'].split('|')[0]])
                test_phrases.append(ent['text'])
            except KeyError:
                log_markup_errors.append({'review_id': review['meta']['fileName'], 'entity_id': ent['xmiID']})
                
USE_CONCEPT_LESS = False
for review in X_test:
    for ent in review['objects']['MedEntity']:
        if 'MedDRA' in ent.keys():
            if ent['MedDRA']=='':
                if USE_CONCEPT_LESS:
                    ent['MedDRA'] = 'CONCEPT_LESS'
                else:
                    continue
            try:
                test_concepts_without_conceptless.append(CV_test_without_conceptless.meddra_term_to_meddra_code[ent['MedDRA'].split('|')[0]])
                test_phrases_without_conceptless.append(ent['text'])
            except KeyError:
                #log_markup_errors.append({'review_id': review['meta']['fileName'], 'entity_id': ent['xmiID']})
                pass
            
print('Всего фраз в трейне: %s'%len(train_phrases))
print('Всего фраз в тесте: %s'%len(test_phrases))

print('Уникальных фраз в трейне: %s'%len(set(train_phrases)))
print('Уникальных фраз в тесте: %s'%len(set(test_phrases)))

#Посмотрим на статистику разбиения
print('%s концептов не входящих либо в трейн, либо в тест'%len(set.union(set(train_concepts), set(test_concepts)) - set.intersection(set(test_concepts), set(train_concepts))))
print('%s концептов, которые есть в тесте, но нет в трейне'%len(set(test_concepts) - set(train_concepts)))
print('%s концептов, которые есть в трейне, но нет в тесте'%len(set(train_concepts) - set(test_concepts)))

Всего фраз в трейне: 8104
Всего фраз в тесте: 3999
Уникальных фраз в трейне: 4365
Уникальных фраз в тесте: 2401
539 концептов не входящих либо в трейн, либо в тест
145 концептов, которые есть в тесте, но нет в трейне
394 концептов, которые есть в трейне, но нет в тесте


In [8]:
from dataset import MedNormDataset

RDR_train = MedNormDataset(train_phrases, train_concepts, CV_train, use_cuda=True)
RDR_test = MedNormDataset(test_phrases, test_concepts, CV_test, use_cuda=True)
RDR_test_without_concepless = MedNormDataset(test_phrases_without_conceptless, test_concepts_without_conceptless, CV_test_without_conceptless, use_cuda=True)

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


тест модельки

In [2]:
import torch.nn as nn
import torch.nn.functional as F
import torch
from transformers import AutoConfig, AutoModel
from collections import UserDict

class CADEC_SoTa_output(UserDict):
    def __init__(self, output):
        if type(output)==dict:
            super(CADEC_SoTa_output, self).__init__(output)
        else:
            super(CADEC_SoTa_output, self).__init__()
            self.data['output'] = output
            self.data['has_padding_for_conceptless_class'] = False
    def to(self, device: str):
        self.data = {k: v.to(device=device) for k, v in self.data.items()}
    def pad_output(self):
        #нулевой класс должен быть под индексом 1
        self.data['output'] = torch.cat((torch.zeros((*(self.data['output'].size()[:-1]), 1), \
                               device = self.data['output'].device), self.data['output']), dim=-1)
        self.data['has_padding_for_conceptless_class'] = True     
    def delete_padding(self):
        self.data['output'] = self.data['output'][:,1:]
        self.data['has_padding_for_conceptless_class'] = False        
    def compute_scores(self):
        self.data['max_scores'] = torch.max(self.data['output'], dim=-1)[0]
    def mask_conceptless(self, score_treshold):
        self.compute_scores()
        concept_exists_term_mask = self.data['max_scores'][:]>score_treshold
        concept_less_term_mask = ~concept_exists_term_mask
        return concept_less_term_mask
    def label_concepless_tensors(self, score_treshold):
        self.pad_output()
        concept_less_tensor = torch.zeros(self.data['output'].size()[-1], dtype=self.data['output'].dtype, device=self.data['output'].device)
        concept_less_tensor[0]=1
        self.data['output'][self.mask_conceptless(score_treshold)] = concept_less_tensor

class CADEC_SoTa(nn.Module):
    def __init__(self, model_path: str, thesaurus_embeddings: torch.tensor):
        super(CADEC_SoTa, self).__init__()
        self.transformer = AutoModel.from_pretrained(model_path)
        self.thesaurus_len, self.hidden_state_size = thesaurus_embeddings.size()
        self.thesaurus_normalized_embs = nn.Parameter(self._normalize_embeddings(thesaurus_embeddings), requires_grad=False)
        
        
    def forward(self, x):
        transformer_inp = {k:v for k,v in x.items() if k in ['input_ids', 'token_type_ids', 'attention_mask']}
        emb = self.transformer(**transformer_inp)
        term_mask = transformer_inp['attention_mask'] if 'input_phrases_masks' not in x.keys() else x['input_phrases_masks']
        x = self._mean_pooling(emb, term_mask)
        #имеем две матрицы x - (batch_size, emb_size) и thesaurus_embeddings - (thesaurus_size, emb_size)
        #надо посчитать косинусную близость близость между каждым вектором x и каждым вложением из тезауруса
        #решение: https://stackoverflow.com/questions/50411191/how-to-compute-the-cosine-similarity-in-pytorch-for-all-rows-in-a-matrix-with-re
        x_n = x.norm(dim=1)[:, None] 
        x_n = x / torch.clamp(x_n, min=1e-8)
        cos_sim = torch.mm(x_n, self.thesaurus_normalized_embs)
        x = F.softmax(cos_sim, dim=1)
        return CADEC_SoTa_output(x)

    
    def _normalize_embeddings(self, emb):
        normalized_embs = emb.norm(dim=1)[:, None]
        normalized_embs = emb / torch.clamp(normalized_embs, min=1e-8)
        normalized_embs = normalized_embs.transpose(0, 1)
        return normalized_embs
    
    def _mean_pooling(self, model_output, attention_mask):
        token_embeddings = model_output[0] #First element of model_output contains all token embeddings
        input_mask_expanded = attention_mask.unsqueeze(-1).expand(token_embeddings.size()).float()
        return torch.sum(token_embeddings * input_mask_expanded, 1) / torch.clamp(input_mask_expanded.sum(1), min=1e-8)



#как инициализировать модель
#модель path - путь до трансформер модели
#CV.thesaurus_embeddings - векторизованный словарь меддры

#import torch

#net = Net(model_path, CV.thesaurus_embeddings)
#device = 'cuda' if torch.cuda.is_available() else 'cpu'
#net.to(device)
#print('Net loaded')


model_path = 'DeepPavlov/rubert-base-cased'
net = CADEC_SoTa(model_path, CV_train.thesaurus_embeddings) #, score_threshold=6.1977e-05
device = 'cuda' if torch.cuda.is_available() else 'cpu'
net.to(device)
print('Net loaded')

NameError: name 'CV_train' is not defined

Демонстрация работы с concept_less

In [1]:
from transformers import AutoTokenizer

net.eval()
phrase = 'сонливость'
model_path = 'DeepPavlov/rubert-base-cased'
tokenizer = AutoTokenizer.from_pretrained(model_path)
encoded_input = tokenizer([phrase], padding=True, truncation=True, return_tensors='pt')

with torch.no_grad():
    outputs_dict = net(encoded_input.to('cuda'))
    outputs_dict.label_concepless_tensors(score_treshold=6.1977e-05)
    pred_meddra_code = CV_test.meddra_codes[outputs_dict['output'].argmax()]
    
print('phrase: %s'%phrase)
print('model: %s'%CV_test.meddra_code_to_meddra_term[pred_meddra_code])

NameError: name 'net' is not defined

<h2>Обучение модели</h2>

In [10]:
import torch.optim as optim
import torch
import numpy as np

criterion = nn.CrossEntropyLoss()
optimizer = optim.AdamW(net.parameters(), lr=0.0001)
#optimizer2 = optim.AdamW(net2.parameters())

In [11]:
scaler = torch.cuda.amp.GradScaler()

Для большей детерменированности

In [26]:
import os 

os.environ["CUBLAS_WORKSPACE_CONFIG"]=":16:8"
torch.use_deterministic_algorithms(mode=False)
np.random.seed(0)
torch.manual_seed(0)
torch.backends.cudnn.benchmark = False

In [13]:
from tqdm import trange
from tqdm import tqdm
from sklearn.metrics import classification_report, f1_score

batch_size=16
trainloader = torch.utils.data.DataLoader(RDR_train, batch_size=batch_size,
                                          shuffle=False, num_workers=0)
testloader = torch.utils.data.DataLoader(RDR_test_without_concepless, batch_size=1, shuffle=False)

net.train()
initial_loss = None
for epoch in range(1, 2):
    net.train()
    with tqdm(trainloader, unit="batch") as tepoch:
        for data in tepoch:

            tepoch.set_description(f"Epoch {epoch}")

            inputs = data['tokenized_phrases']
            labels = data['one_hot_labels']

            optimizer.zero_grad()
            if device=='cuda':
                with torch.cuda.amp.autocast():
                    outputs = net(inputs)['output']
                    loss = criterion(outputs, labels)
                scaler.scale(loss).backward()
                scaler.step(optimizer)
                scaler.update()
            else:
                outputs = net(inputs)['output']
                loss = criterion(outputs, labels)
                loss.backward()
                optimizer.step()
            if initial_loss is None:
                initial_loss = loss.item()
            tepoch.set_postfix(loss_decrease = str(initial_loss/loss.item()))
    #net.eval()
    #model_answers=[]
    #real_answers=[]
    #with tqdm(testloader, unit="batch") as eval_process:
    #    for data in eval_process:

            #tepoch.set_description(f"Progress")

    #        inputs = data['tokenized_phrases']
            #labels = data['one_hot_labels']

    #        with torch.no_grad():
    #            outputs_dict = net(inputs)
                #outputs_dict.label_concepless_tensors(score_treshold = 6.1977e-05)
    #            pred_meddra_code = CV_test_without_conceptless.meddra_codes[outputs_dict['output'].argmax()]


     #       model_answers.append(pred_meddra_code)
     #       real_answers.append(data['label_codes'])

    #print(f1_score(real_answers, model_answers, average='micro'))
            
print('Finished Training')

Epoch 1: 100%|██████████| 507/507 [01:12<00:00,  6.97batch/s, loss_decrease=1.0000015131880964]

Finished Training


In [25]:
net.transformer.save_pretrained('./rubert_trained_1ep_debug/')

In [15]:
torch.save(net, './rubert_trained_1ep_debug/rubert_1ep_best_nocontext.pt')
torch.save(optimizer, './rubert_trained_1ep_debug/rubert_1ep_best_nocontext_opt.pt')

In [ ]:
#torch.save(net, './rubert_trained_1ep_debug/rubert_1ep_best_nocontext.pt')
#torch.save(optimizer, './rubert_trained_1ep_debug/rubert_1ep_best_nocontext_opt.pt')

In [18]:
#from transformers import AutoTokenizer

net.eval()
#phrase = 'Недостаточность 11-бета-гидроксилазы'

CV_with_rubert_trained = ConceptVectorizer('DeepPavlov/rubert-base-cased', '../../Data/External/pt_rus.asc', \
                             use_concept_less=False, use_model = False, use_cuda=True)


#model_path = 'DeepPavlov/rubert-base-cased'
#tokenizer = AutoTokenizer.from_pretrained(model_path)
encoded_input = CV_with_rubert_trained.tokenizer(list(CV_with_rubert_trained.meddra_term_to_meddra_code.keys())[0], \
                          padding=True, truncation=True, return_tensors='pt')
encoded_input = encoded_input.to('cuda')

with torch.no_grad():
    print(CV_with_rubert_trained.mean_pooling(net.transformer(**encoded_input)[0], encoded_input['attention_mask']))

Some weights of the model checkpoint at DeepPavlov/rubert-base-cased were not used when initializing BertModel: ['cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum lengt

tensor([[ 3.9090e-01, -3.6601e-01, -7.0845e-02, -8.0149e-01,  1.0696e+00,
          3.1359e-01, -1.8173e-02, -7.2493e-01,  2.9128e-02, -3.2770e-01,
         -2.3398e-01, -7.2678e-01,  4.4805e-01,  4.6029e-01,  1.2352e+00,
         -9.7021e-01,  1.6440e-01, -3.3506e-01, -6.6074e-02, -5.9675e-02,
         -3.4805e-01, -8.7446e-04,  5.9289e-01, -2.8026e-01, -2.5854e-01,
          4.0817e-01,  8.1454e-01,  3.8441e-01, -5.7781e-01, -2.9541e-01,
         -2.1116e-01, -1.0536e+00, -1.3720e+00, -3.2537e-01,  4.7316e-01,
          3.2409e-01, -6.0984e-01, -1.6708e-01,  5.7921e-01, -4.2840e-01,
         -1.0331e-02,  1.3920e+00, -6.0062e-01,  3.3385e-01,  3.1305e-01,
          2.6414e-01,  1.9540e-01,  7.7554e-01,  4.4389e-01,  4.6227e-02,
          3.3317e-01, -3.7784e-01, -1.9856e-01, -5.2241e-01, -4.1026e-01,
         -2.7379e-01, -1.7067e-02,  4.4213e-01, -1.8394e-01, -6.3743e-01,
         -1.4415e-01, -1.3950e-01,  5.0619e-01,  4.7866e-01,  1.3588e-01,
          7.1954e-01, -2.3333e-01, -1.

In [23]:
for param in net.transformer.parameters():
    print(param)

Parameter containing:
tensor([[ 0.0041,  0.0137, -0.0080,  ...,  0.0170,  0.0175,  0.0226],
        [ 0.0010,  0.0184,  0.0111,  ..., -0.0029, -0.0008,  0.0019],
        [ 0.0025,  0.0105,  0.0073,  ..., -0.0028, -0.0003, -0.0066],
        ...,
        [ 0.0035,  0.0057,  0.0148,  ..., -0.0182, -0.0150, -0.0011],
        [-0.0342,  0.0004, -0.0859,  ..., -0.0019, -0.0255, -0.0084],
        [ 0.0178,  0.0073, -0.0294,  ...,  0.0108,  0.0266,  0.0045]],
       device='cuda:0', requires_grad=True)
Parameter containing:
tensor([[ 0.1172,  0.0938,  0.0253,  ..., -0.0565, -0.0333,  0.0429],
        [-0.0184, -0.0107,  0.0213,  ...,  0.0261, -0.0058, -0.0070],
        [-0.0120, -0.0086,  0.0180,  ...,  0.0129,  0.0113, -0.0130],
        ...,
        [ 0.0122,  0.0608, -0.0022,  ..., -0.0138, -0.0029,  0.0346],
        [ 0.0487,  0.0351,  0.0228,  ..., -0.0046, -0.0249,  0.0398],
        [ 0.0070,  0.0584, -0.0283,  ..., -0.0122, -0.0294,  0.0412]],
       device='cuda:0', requires_grad=True)


Parameter containing:
tensor([-6.3340e-03,  4.4528e-02, -5.0024e-02,  1.9900e-01,  3.4924e-02,
         1.8948e-02,  2.0604e-02,  2.4508e-02,  5.1868e-02,  5.8441e-02,
         2.0303e-02, -5.3981e-03, -4.2039e-02, -1.4336e-02, -2.1316e-02,
        -4.3298e-02, -5.5845e-02, -6.1106e-02,  4.6557e-02,  3.8390e-02,
        -9.6166e-03,  8.9178e-02, -3.7626e-02, -4.3052e-02,  1.0200e-01,
        -6.0963e-02, -8.4872e-03, -2.4983e-02, -3.9581e-02, -6.3248e-02,
         5.0876e-02,  3.2368e-02, -2.3316e-03, -1.7734e-02, -3.9518e-02,
         7.1939e-02,  4.9247e-02, -1.4466e-02, -8.7897e-02, -2.4330e-02,
        -6.4143e-02,  7.5180e-03,  2.5336e-02,  1.8823e-01, -9.5824e-02,
         3.8223e-02,  5.2559e-02,  2.7957e-02, -1.2252e-02,  1.4140e-02,
         7.2573e-02,  1.4358e-02,  1.8978e-02, -8.5214e-03, -3.7150e-02,
        -1.3300e-02,  2.6093e-02,  2.2542e-02, -2.4623e-02,  1.9736e-03,
        -1.3349e-01,  3.8567e-02, -1.8937e-01,  1.7391e-02,  1.5683e-02,
         3.0202e-04, -9.6400e

Parameter containing:
tensor([-5.9923e-02,  2.5452e-01,  6.4912e-03,  9.7790e-02,  5.7674e-02,
         3.4521e-02, -3.4776e-02, -1.5253e-02,  7.7746e-02,  5.4217e-02,
        -2.2496e-02,  1.9683e-03,  8.8533e-02,  3.7793e-02,  7.2476e-02,
        -7.4792e-02,  6.1842e-02, -9.4514e-03, -1.2394e-01,  1.1810e-01,
         3.1494e-02, -9.9343e-03, -4.9455e-02, -2.7468e-02,  1.0222e-01,
        -4.8923e-02,  7.0607e-02, -4.4021e-02, -1.3375e-03,  6.9872e-02,
        -4.3851e-02,  1.3772e-02,  4.8372e-02, -9.6183e-02,  5.4966e-02,
         8.1022e-02,  7.1935e-02,  1.4219e-01,  4.6963e-02,  7.9496e-02,
         2.0804e-02, -9.7881e-02,  1.6508e-01, -2.7102e-02, -7.5942e-02,
         1.1542e-01, -3.5619e-02,  1.1981e-02,  2.0737e-02,  1.8569e-02,
        -7.2922e-02,  2.0592e-02,  6.2253e-02, -6.7273e-02,  2.4873e-01,
        -4.7570e-02,  3.8138e-02, -2.1492e-02,  4.8743e-02,  1.5440e-01,
         8.2372e-02, -5.7547e-02, -7.4929e-03,  9.2136e-02,  2.6775e-01,
        -5.0297e-02, -1.7871e

Parameter containing:
tensor([[ 0.0566, -0.0009, -0.0575,  ...,  0.0449, -0.0146, -0.0097],
        [-0.1184, -0.0241, -0.0191,  ...,  0.0271,  0.0543, -0.0307],
        [ 0.0856,  0.0452, -0.0330,  ...,  0.0521,  0.0191, -0.0583],
        ...,
        [ 0.0470,  0.0513,  0.0136,  ...,  0.0169,  0.0019, -0.0116],
        [-0.0285,  0.0061, -0.0236,  ...,  0.0177,  0.0513,  0.0359],
        [-0.0624, -0.0398, -0.0126,  ...,  0.0135,  0.0105,  0.0027]],
       device='cuda:0', requires_grad=True)
Parameter containing:
tensor([-1.1888e-02,  3.6114e-03, -1.0174e-02,  2.4298e-03,  1.4589e-02,
         5.8977e-03, -5.5661e-03, -1.6354e-03,  5.9219e-03,  1.1680e-02,
        -5.7681e-03,  4.8109e-03,  8.3628e-03,  3.4441e-03,  1.6914e-04,
         2.3751e-03,  1.8971e-03, -3.8127e-04,  3.6728e-03, -8.7129e-03,
        -7.0306e-04, -5.9987e-03, -4.8676e-03, -6.3837e-03, -5.9033e-03,
        -9.2827e-03, -6.8774e-03,  2.4671e-03,  3.4181e-03,  3.6374e-05,
        -7.9164e-03, -2.5673e-03,  5.133

Parameter containing:
tensor([-4.6390e-02, -2.9556e-02,  2.6785e-02, -1.1347e-01,  5.0258e-02,
        -8.0078e-02,  5.7439e-02,  2.1303e-02, -4.6057e-02,  4.1960e-02,
         2.0305e-02,  9.6069e-02, -1.3781e-01,  1.9501e-02,  1.9830e-02,
        -1.9618e-02,  4.1173e-02,  4.2268e-02,  9.2102e-02,  1.2928e-02,
        -8.4842e-02, -6.7724e-02,  9.7355e-02, -2.0422e-02, -7.5764e-03,
        -5.5145e-03, -1.4549e-01, -8.4384e-02,  7.4182e-03, -8.7434e-02,
         1.0786e-01,  1.0483e-01, -9.6539e-02,  6.6144e-02, -1.6731e-01,
         3.2733e-02, -5.6923e-03, -7.6262e-02,  4.1555e-03, -2.0917e-03,
        -5.6088e-02, -5.2119e-02, -4.9778e-02,  2.0820e-02,  2.1547e-01,
         9.1478e-02,  9.6019e-02,  5.8191e-03, -9.4714e-02, -7.6158e-03,
         1.4649e-02,  1.0758e-01,  5.4378e-03,  3.6015e-02, -1.5584e-01,
        -1.6814e-01, -7.9973e-02, -5.1112e-02, -6.4900e-03, -8.4955e-02,
        -1.8247e-02, -3.8442e-02, -3.9698e-02,  1.3943e-01, -3.7859e-02,
         3.6020e-02, -3.8981e

...

Parameter containing:
tensor([-2.1671e-02, -5.2963e-03, -3.8587e-02,  8.2132e-02, -6.6395e-02,
        -3.3763e-02,  3.3714e-02, -3.4209e-02,  1.4662e-02, -1.0749e-01,
         7.6482e-02, -1.1123e-01,  2.0295e-02,  2.0011e-03, -4.1317e-02,
        -9.0929e-02, -9.1784e-02, -3.6980e-02,  6.6765e-02, -6.5336e-02,
         5.7434e-02,  1.0926e-01, -1.2362e-01,  6.5924e-02,  2.1324e-03,
        -3.2242e-02, -5.9069e-02,  4.7930e-02, -2.3556e-02,  9.8749e-02,
         3.5744e-02,  4.6083e-02,  1.8476e-01, -1.5772e-01, -1.5653e-01,
        -5.5273e-02, -3.7513e-02, -1.1771e-01,  7.0283e-02,  2.3427e-03,
         5.2957e-02,  6.5260e-02,  6.8136e-02, -1.0063e-02,  6.4603e-02,
         1.5469e-02,  3.6583e-02,  5.6808e-02,  3.0262e-02,  3.6943e-02,
        -9.2202e-02,  1.5463e-02,  7.2632e-02,  2.3786e-02,  1.3291e-01,
         7.8271e-02,  3.9928e-03,  8.4074e-02,  3.9842e-03, -1.4243e-01,
        -1.0634e-01, -1.7526e-02, -1.3427e-01, -7.5239e-02,  6.8863e-02,
         8.7816e-02, -2.3489e

Parameter containing:
tensor([0.9547, 0.8939, 0.9662, 0.8989, 0.9638, 0.9754, 0.9461, 0.9204, 0.9826,
        0.9938, 0.9951, 0.9704, 0.9707, 0.9644, 0.9245, 0.9462, 0.9869, 0.9730,
        0.9605, 0.9518, 0.9562, 0.9094, 0.9835, 0.9735, 0.9804, 0.9835, 0.9860,
        0.9703, 0.9831, 0.9176, 0.9486, 0.9688, 0.9061, 0.9529, 0.8478, 0.9811,
        0.9506, 0.9687, 0.9646, 1.0077, 0.9743, 0.9659, 0.9804, 0.8606, 0.8917,
        0.9045, 0.9355, 0.9593, 0.9492, 0.9889, 0.9994, 0.9052, 1.0029, 0.9671,
        0.9722, 0.9445, 0.9976, 0.9730, 0.9383, 0.9686, 0.9107, 0.9655, 0.9054,
        0.9620, 0.9731, 0.9596, 0.9716, 0.8746, 0.9832, 0.9723, 1.0106, 0.9273,
        0.9499, 0.8574, 0.9687, 0.9929, 0.9060, 0.9808, 0.9908, 0.9918, 0.9252,
        0.9724, 0.9958, 0.9540, 0.9344, 0.9506, 0.9671, 0.9551, 0.9192, 0.9744,
        1.0050, 0.9652, 0.9513, 0.9452, 0.9252, 0.8991, 0.9606, 0.9077, 0.9282,
        0.9405, 0.9457, 0.9809, 0.9941, 0.8624, 0.9484, 0.9672, 0.9812, 0.9830,
        1.0118, 0.

Parameter containing:
tensor([[-0.0502, -0.0311,  0.0502,  ..., -0.0503, -0.0375,  0.0189],
        [ 0.0063, -0.0332,  0.0874,  ...,  0.0562, -0.0380,  0.0003],
        [-0.0014, -0.0169,  0.0203,  ...,  0.0238,  0.0382,  0.0732],
        ...,
        [ 0.0310, -0.0075, -0.0812,  ..., -0.0062,  0.0147,  0.0651],
        [ 0.0180, -0.0496,  0.0039,  ...,  0.0887, -0.0004, -0.0207],
        [ 0.0208, -0.0729,  0.0189,  ..., -0.0127,  0.0154,  0.0240]],
       device='cuda:0', requires_grad=True)
Parameter containing:
tensor([ 5.3153e-02,  2.4665e-01, -3.5050e-01, -6.7387e-02, -2.3188e-01,
         1.0342e-01,  2.4394e-02, -2.2387e-02,  1.6880e-01, -1.7163e-01,
        -8.5863e-02,  1.5037e-01, -1.3186e-01,  1.5961e-01,  4.3494e-01,
         5.5899e-02, -2.2471e-01,  1.2187e-01, -4.3608e-01, -2.6678e-02,
        -1.8061e-01,  1.1700e-01, -1.8677e-02,  9.6947e-02, -2.2725e-01,
         3.6134e-01,  7.5826e-02, -9.6038e-02,  2.8412e-01, -1.0375e-01,
         2.9018e-01,  6.3559e-02,  2.855

Parameter containing:
tensor([-2.1724e-03,  2.8661e-03,  4.2562e-03, -4.9044e-03, -8.1745e-03,
         6.6316e-04,  5.1372e-03, -3.4342e-03,  2.7919e-04, -5.2461e-03,
        -6.0449e-03,  5.4807e-03, -1.1192e-05, -2.6951e-03, -2.9977e-03,
        -7.0278e-04,  4.7615e-04, -9.1228e-05, -4.7305e-03,  2.3912e-03,
        -1.0468e-02,  3.8895e-03,  2.9659e-03, -1.2379e-03,  5.6827e-03,
        -4.8096e-03,  5.5365e-04,  3.1874e-03, -9.3202e-03, -2.0491e-03,
         4.5823e-03,  6.8368e-03,  8.6423e-04, -5.1612e-04, -4.2305e-03,
         2.4866e-03, -8.7818e-03,  1.3343e-03,  4.9591e-03, -1.7640e-03,
        -2.5110e-03, -2.9883e-03, -1.5686e-03, -1.8440e-03,  1.0883e-03,
        -2.5817e-03, -4.1023e-03,  3.2883e-04,  1.1747e-03, -4.0929e-03,
        -4.7719e-03, -6.0297e-03,  5.9636e-03,  5.4955e-03, -1.5933e-03,
        -9.9255e-03, -6.0012e-03,  1.9890e-03, -1.3982e-03,  3.1239e-03,
         2.3170e-03, -3.5116e-03,  1.3937e-03,  8.2073e-04, -5.3610e-03,
        -4.5273e-03, -9.5678e

Parameter containing:
tensor([0.8846, 0.8909, 0.9564, 0.8749, 0.9000, 0.9413, 0.9333, 0.9383, 0.8901,
        0.9134, 0.9064, 0.9142, 0.9099, 0.8715, 0.9088, 0.9099, 0.8849, 0.9346,
        0.8864, 0.9113, 0.8959, 0.9154, 0.9167, 0.9077, 0.9068, 0.9261, 0.8974,
        0.9075, 0.9126, 0.9212, 0.9182, 0.8874, 0.8929, 0.8916, 0.8734, 0.9398,
        1.0301, 0.8848, 0.9001, 0.9750, 0.9339, 0.9126, 0.9341, 0.8853, 0.8787,
        1.0145, 0.8857, 0.8763, 1.1550, 0.9257, 0.9130, 0.9478, 0.8670, 1.0792,
        0.8925, 0.8940, 0.8836, 0.9379, 0.9236, 0.9043, 0.8676, 0.9447, 0.9255,
        0.8802, 0.8677, 0.9050, 0.8764, 0.8781, 0.9202, 0.9199, 0.9102, 0.8909,
        0.9104, 0.8777, 0.9393, 0.9245, 0.8925, 0.8862, 0.9199, 0.8916, 0.9083,
        0.8816, 0.8947, 0.9208, 0.9112, 0.9023, 0.9526, 0.9024, 0.9519, 0.9484,
        0.9083, 0.9496, 0.9942, 0.9269, 0.9053, 1.0149, 0.8913, 0.8771, 0.8983,
        0.8773, 0.9005, 1.0002, 0.8974, 0.9156, 1.0638, 0.9336, 0.9209, 0.8809,
        0.8932, 0.

In [12]:
CV_with_rubert_trained = ConceptVectorizer('DeepPavlov/rubert-base-cased', '../../Data/External/pt_rus.asc', \
                             use_concept_less=False, use_model=False, use_cuda=True) 
#, use_model=False
#CV_with_rubert_trained.model = net.transformer
#CV_with_rubert_trained.fit_transform(mode='mean_pooling')
CV_with_rubert_trained.thesaurus_embeddings = torch.load('rubert_trained_1ep_thesaurus_embeddings.pt')
CV_with_rubert_trained.normalization_mode = 'mean_pooling'

net_new_concept_weights = CADEC_SoTa('./rubert_trained_1ep_debug/', CV_with_rubert_trained.thesaurus_embeddings)

import os
os.environ["CUBLAS_WORKSPACE_CONFIG"]=":16:8"
torch.use_deterministic_algorithms(mode=False)
np.random.seed(0)
torch.manual_seed(0)
torch.backends.cudnn.benchmark = False

for param in net_new_concept_weights.transformer.parameters():
    print(param)

Some weights of the model checkpoint at DeepPavlov/rubert-base-cased were not used when initializing BertModel: ['cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.predictions.decoder.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Parameter containing:
tensor([[ 0.0041,  0.0137, -0.0080,  ...,  0.0170,  0.0175,  0.0226],
        [ 0.0010,  0.0184,  0.0111,  ..., -0.0029, -0.0008,  0.0019],
        [ 0.0025,  0.0105,  0.0073,  ..., -0.0028, -0.0003, -0.0066],
        ...,
        [ 0.0035,  0.0057,  0.0148,  ..., -0.0182, -0.0150, -0.0011],
        [-0.0342,  0.0004, -0.0859,  ..., -0.0019, -0.0255, -0.0084],
        [ 0.0178,  0.0073, -0.0294,  ...,  0.0108,  0.0266,  0.0045]],
       requires_grad=True)
Parameter containing:
tensor([[ 0.1172,  0.0938,  0.0253,  ..., -0.0565, -0.0333,  0.0429],
        [-0.0184, -0.0107,  0.0213,  ...,  0.0261, -0.0058, -0.0070],
        [-0.0120, -0.0086,  0.0180,  ...,  0.0129,  0.0113, -0.0130],
        ...,
        [ 0.0122,  0.0608, -0.0022,  ..., -0.0138, -0.0029,  0.0346],
        [ 0.0487,  0.0351,  0.0228,  ..., -0.0046, -0.0249,  0.0398],
        [ 0.0070,  0.0584, -0.0283,  ..., -0.0122, -0.0294,  0.0412]],
       requires_grad=True)
Parameter containing:
tensor([[ 0.

Parameter containing:
tensor([0.8941, 0.8093, 0.9203, 0.8436, 0.8918, 0.9039, 0.9191, 0.9132, 0.9249,
        0.9119, 0.9418, 0.9177, 0.9086, 0.9254, 0.8988, 0.8973, 0.9174, 0.9053,
        0.8453, 0.8740, 0.9152, 0.7859, 0.9209, 0.9291, 0.9434, 0.9148, 0.9352,
        0.8994, 0.9161, 0.8869, 0.8924, 0.9175, 0.8503, 0.8930, 0.8429, 0.9000,
        0.9326, 0.9194, 0.8935, 0.9192, 0.8919, 0.9016, 0.9039, 0.9554, 0.8543,
        0.9456, 0.8730, 0.9107, 0.9314, 0.9097, 0.9508, 0.8708, 0.9126, 0.9049,
        0.9565, 0.8572, 0.9147, 0.9600, 0.8790, 0.9048, 0.8772, 0.8990, 0.8759,
        0.8840, 0.8963, 0.8922, 0.9070, 0.8182, 0.9068, 0.8946, 0.9203, 0.8962,
        0.9004, 0.8762, 0.9104, 0.9146, 0.8856, 0.9091, 0.9345, 0.9282, 0.8996,
        0.8931, 0.8908, 0.9069, 0.9226, 0.8970, 0.8983, 0.8964, 0.8816, 0.9166,
        0.9434, 0.9246, 0.8962, 0.8963, 0.8828, 0.8555, 0.8721, 0.8871, 0.7528,
        0.9077, 0.8823, 0.9092, 0.9008, 0.8444, 0.8809, 0.9178, 0.9211, 0.8947,
        0.9353, 0.

Parameter containing:
tensor([ 1.1256e-02,  1.3866e-03, -3.9164e-03,  3.2762e-04, -1.7836e-03,
         6.4357e-04, -1.0178e-03,  3.8756e-03,  4.2641e-03, -3.5056e-04,
        -2.5111e-03,  4.7895e-03, -1.9817e-03, -6.8771e-03,  7.5420e-03,
         3.6436e-04, -2.1854e-03,  1.0139e-03,  9.9568e-05, -1.1108e-03,
         1.5944e-03, -5.2279e-04,  1.8806e-03,  1.0162e-02,  2.2513e-03,
        -2.6835e-03, -2.1845e-03,  5.5695e-03,  2.0712e-03,  2.2962e-03,
        -3.3668e-03,  6.9534e-03, -5.5621e-03, -1.8069e-03, -1.7450e-03,
         1.3402e-03, -8.5841e-03,  3.4585e-04, -1.7658e-03,  1.1596e-03,
         7.9877e-03, -2.9801e-03, -4.8005e-03, -6.6307e-05, -1.6121e-03,
         6.6490e-03,  1.1835e-04, -4.3944e-03,  8.7770e-03,  5.4721e-03,
         1.0324e-03, -4.2296e-03, -1.6617e-03,  2.5418e-03,  1.2979e-03,
         8.9685e-05,  4.3034e-03,  2.6731e-03,  2.7789e-03,  2.6456e-03,
         1.6387e-03, -6.5423e-04,  6.0354e-03,  8.2770e-03, -4.4112e-03,
         7.1627e-03,  1.2377e

Parameter containing:
tensor([0.9711, 0.9249, 0.9778, 0.9403, 0.9774, 0.9947, 0.9625, 0.9562, 0.9817,
        0.9940, 0.9964, 0.9875, 0.9686, 0.9866, 0.9581, 0.9853, 1.0075, 0.9929,
        0.9866, 0.9747, 0.9801, 0.9353, 1.0120, 0.9844, 0.9962, 0.9944, 0.9877,
        0.9913, 0.9893, 0.9565, 0.9804, 0.9891, 0.9661, 0.9804, 0.8924, 0.9839,
        0.9702, 1.0146, 1.0242, 1.0224, 1.0071, 0.9684, 0.9542, 0.9269, 0.9254,
        0.9281, 0.9667, 0.9996, 0.9183, 0.9948, 1.0341, 0.9176, 0.9740, 0.9802,
        0.9864, 0.9628, 0.9903, 0.9992, 0.9656, 0.9757, 0.9505, 0.9827, 0.9419,
        0.9668, 0.9778, 0.9786, 0.9928, 0.9147, 0.9848, 0.9828, 0.9920, 0.9705,
        0.9436, 0.9215, 0.9934, 0.9893, 0.9468, 0.9878, 1.0005, 1.0043, 0.9714,
        0.9825, 1.0129, 0.9685, 0.9789, 0.9727, 0.9819, 0.9820, 0.9710, 1.0104,
        1.0307, 1.0004, 0.9590, 0.9968, 0.9248, 0.9346, 0.9869, 0.9232, 0.9466,
        0.9750, 0.9394, 1.0024, 0.9866, 0.9152, 0.9825, 0.9861, 0.9959, 0.9847,
        0.9966, 0.

In [25]:
torch.save(CV_with_rubert_trained.thesaurus_embeddings, 'rubert_trained_1ep_thesaurus_embeddings.pt')

In [15]:
from tqdm import tqdm
from sklearn.metrics import f1_score
#print('Net loaded')

net_new_concept_weights.eval()
model_answers=[]
real_answers=[]
model_terms=[]
real_terms=[]
testloader = torch.utils.data.DataLoader(RDR_test_without_concepless, batch_size=1, shuffle=False)
with tqdm(testloader, unit="batch") as eval_process:
    for data in eval_process:

        #tepoch.set_description(f"Progress")

        inputs = data['tokenized_phrases']
        #labels = data['one_hot_labels']

        with torch.no_grad():
            outputs_dict = net_new_concept_weights(inputs)
            #outputs_dict.label_concepless_tensors(score_treshold = 6.1977e-05)
            pred_meddra_code = CV_test_without_conceptless.meddra_codes[outputs_dict['output'].argmax()]


        model_answers.append(pred_meddra_code)
        real_answers.append(data['label_codes'])
        model_terms.append(data[''])
        real_terms.append(data[''])

print(f1_score(real_answers, model_answers, average='micro'))

Net loaded


100%|██████████| 3780/3780 [00:48<00:00, 77.17batch/s]

0.3637566137566138


In [18]:
net = CADEC_SoTa('./rubert_trained_1ep_debug/', CV_train.thesaurus_embeddings)
print('net loaded')
net.eval()
net.to('cuda')

import os 
os.environ["CUBLAS_WORKSPACE_CONFIG"]=":16:8"
torch.use_deterministic_algorithms(mode=False)
np.random.seed(0)
torch.manual_seed(0)
torch.backends.cudnn.benchmark = False

model_answers=[]
real_answers=[]
with tqdm(testloader, unit="batch") as eval_process:
    for data in eval_process:


        inputs = data['tokenized_phrases']
        labels = data['one_hot_labels']

        with torch.no_grad():
            outputs_dict = net(inputs)
            #outputs_dict.label_concepless_tensors(score_treshold = 6.1977e-05)
            pred_meddra_code = CV_test_without_conceptless.meddra_codes[outputs_dict['output'].argmax()]


            model_answers.append(pred_meddra_code)
            real_answers.append(data['label_codes'])

print(f1_score(real_answers, model_answers, average='micro'))

net loaded


100%|██████████| 3780/3780 [00:44<00:00, 84.09batch/s]

0.6044973544973545


In [23]:
net_new_concept_weights.eval()
encoded_input = CV_train.tokenizer(list(CV_train.meddra_term_to_meddra_code.keys())[0], padding=True, truncation=True, return_tensors='pt')
encoded_input.to('cuda')
with torch.no_grad():
    out = net_new_concept_weights.transformer(**encoded_input)
print(F.normalize(CV_train.mean_pooling(out[0], encoded_input['attention_mask'])))

tensor([[ 2.7553e-02, -2.5798e-02, -4.9936e-03, -5.6494e-02,  7.5389e-02,
          2.2104e-02, -1.2809e-03, -5.1097e-02,  2.0531e-03, -2.3098e-02,
         -1.6492e-02, -5.1227e-02,  3.1581e-02,  3.2444e-02,  8.7061e-02,
         -6.8386e-02,  1.1588e-02, -2.3617e-02, -4.6573e-03, -4.2062e-03,
         -2.4532e-02, -6.1637e-05,  4.1790e-02, -1.9754e-02, -1.8223e-02,
          2.8770e-02,  5.7413e-02,  2.7095e-02, -4.0727e-02, -2.0822e-02,
         -1.4884e-02, -7.4266e-02, -9.6703e-02, -2.2934e-02,  3.3351e-02,
          2.2843e-02, -4.2984e-02, -1.1777e-02,  4.0826e-02, -3.0196e-02,
         -7.2818e-04,  9.8116e-02, -4.2335e-02,  2.3532e-02,  2.2066e-02,
          1.8618e-02,  1.3773e-02,  5.4664e-02,  3.1288e-02,  3.2583e-03,
          2.3484e-02, -2.6633e-02, -1.3995e-02, -3.6822e-02, -2.8917e-02,
         -1.9298e-02, -1.2030e-03,  3.1163e-02, -1.2965e-02, -4.4930e-02,
         -1.0160e-02, -9.8329e-03,  3.5679e-02,  3.3738e-02,  9.5776e-03,
          5.0717e-02, -1.6447e-02, -1.

In [24]:
torch.transpose(net_new_concept_weights.thesaurus_normalized_embs, 1, 0)[0]

tensor([ 2.7553e-02, -2.5798e-02, -4.9936e-03, -5.6494e-02,  7.5389e-02,
         2.2104e-02, -1.2809e-03, -5.1097e-02,  2.0531e-03, -2.3098e-02,
        -1.6492e-02, -5.1227e-02,  3.1581e-02,  3.2444e-02,  8.7061e-02,
        -6.8386e-02,  1.1588e-02, -2.3617e-02, -4.6573e-03, -4.2062e-03,
        -2.4532e-02, -6.1635e-05,  4.1790e-02, -1.9754e-02, -1.8223e-02,
         2.8770e-02,  5.7413e-02,  2.7095e-02, -4.0727e-02, -2.0822e-02,
        -1.4884e-02, -7.4266e-02, -9.6703e-02, -2.2934e-02,  3.3351e-02,
         2.2843e-02, -4.2984e-02, -1.1777e-02,  4.0826e-02, -3.0196e-02,
        -7.2818e-04,  9.8116e-02, -4.2335e-02,  2.3532e-02,  2.2066e-02,
         1.8618e-02,  1.3773e-02,  5.4664e-02,  3.1288e-02,  3.2584e-03,
         2.3484e-02, -2.6633e-02, -1.3995e-02, -3.6822e-02, -2.8918e-02,
        -1.9298e-02, -1.2030e-03,  3.1163e-02, -1.2965e-02, -4.4930e-02,
        -1.0160e-02, -9.8329e-03,  3.5679e-02,  3.3738e-02,  9.5776e-03,
         5.0717e-02, -1.6447e-02, -1.1270e-02, -5.3

In [26]:
net_new_concept_weights.eval()
phrase = 'Недостаточность 11-бета-гидроксилазы'
#list(CV_train.meddra_term_to_meddra_code.keys())[0]

with torch.no_grad():
    encoded_input = CV_train.tokenizer([phrase], padding=True, truncation=True, return_tensors='pt')
    encoded_input.to('cuda')
    print(net_new_concept_weights(encoded_input)['output'].argmax())

tensor(0, device='cuda:0')


In [ ]:
net_new_concept_weights.thesaurus_embeddings[0]

In [ ]:
from transformers import AutoTokenizer

model_path = 'DeepPavlov/rubert-base-cased'
tokenizer = AutoTokenizer.from_pretrained(model_path)
tokenizer.save_pretrained('./rubert_trained_1ep_debug/')

In [19]:
!ls ./rubert_trained_1ep_debug

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
rubert_1ep_best_nocontext_opt.pt  special_tokens_map.json  tokenizer.json
rubert_1ep_best_nocontext.pt	  tokenizer_config.json    vocab.txt


In [15]:
torch.save(net, './rubert_9ep_best_nocontext.pt')
torch.save(optimizer, './rubert_9ep_best_nocontext_opt.pt')

<h3>Эксперимент, запрошенный Сбоевым, с весами encoder'a концептов</h3>

In [54]:
from transformers import AutoTokenizer
from models import CADEC_SoTa
from vectorization import ConceptVectorizer
import torch
from tqdm import tqdm
from sklearn.metrics import f1_score
#net.transformer.save_pretrained('./rubert_trained_9ep_no_Context/')

#model_path = 'DeepPavlov/rubert-base-cased'
#tokenizer = AutoTokenizer.from_pretrained(model_path)
#tokenizer.save_pretrained('./rubert_trained_9ep_no_Context/')

CV_with_rubert_trained = ConceptVectorizer('./rubert_trained_1ep_debug/', '../../Data/External/pt_rus.asc', \
                             use_concept_less=False, use_cuda=True) #, use_model=False

CV_with_rubert_trained.fit_transform(mode='mean_pooling')

#CV_with_rubert_trained.thesaurus_embeddings = torch.load('thesaurus_emb_trained_9ep_noContext_rubert.pt')
#CV_with_rubert_trained.vectorization_mode = 'mean_pooling' 

model_path = 'DeepPavlov/rubert-base-cased'
net_new_concept_weights = CADEC_SoTa('./rubert_trained_1ep_debug/', CV_with_rubert_trained.thesaurus_embeddings) #, score_threshold=6.1977e-05
device = 'cuda' if torch.cuda.is_available() else 'cpu'
net_new_concept_weights.to(device)
print('Net loaded')

net_new_concept_weights.eval()
model_answers=[]
real_answers=[]
testloader = torch.utils.data.DataLoader(RDR_test_without_concepless, batch_size=1, shuffle=False)
with tqdm(testloader, unit="batch") as eval_process:
    for data in eval_process:

        #tepoch.set_description(f"Progress")

        inputs = data['tokenized_phrases']
        #labels = data['one_hot_labels']

        with torch.no_grad():
            outputs_dict = net_new_concept_weights(inputs)
            #outputs_dict.label_concepless_tensors(score_treshold = 6.1977e-05)
            pred_meddra_code = CV_test_without_conceptless.meddra_codes[outputs_dict['output'].argmax()]


        model_answers.append(pred_meddra_code)
        real_answers.append(data['label_codes'])

print(f1_score(real_answers, model_answers, average='micro'))

loading model...


Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


getting concept embeddings in mean_pooling mode...
moving to cuda... device name cuda:0
Compute embeddings...


RuntimeError: CUDA out of memory. Tried to allocate 10.14 GiB (GPU 0; 31.75 GiB total capacity; 20.06 GiB already allocated; 2.24 GiB free; 28.17 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

In [2]:
model_path = 'DeepPavlov/rubert-base-cased'
tokenizer = AutoTokenizer.from_pretrained(model_path)
tokenizer(['fhhf'])

{'input_ids': [[101, 242, 247, 247, 243, 102]], 'token_type_ids': [[0, 0, 0, 0, 0, 0]], 'attention_mask': [[1, 1, 1, 1, 1, 1]]}

In [18]:
torch.save(CV_with_rubert_trained.thesaurus_embeddings, 'thesaurus_emb_trained_9ep_noContext_rubert.pt')

<h2>Тест модели с mean_pooling c CONCEPT_LESS</h2>

In [14]:
from sklearn.metrics import classification_report, f1_score
from tqdm import tqdm


net.eval()

model_answers=[]
real_answers=[]

testloader = torch.utils.data.DataLoader(RDR_test, batch_size=1,
                                          shuffle=False)


with tqdm(testloader, unit="batch") as eval_process:
    for data in eval_process:

        #tepoch.set_description(f"Progress")

        inputs = data['tokenized_phrases']
        #labels = data['one_hot_labels']

        with torch.no_grad():
            outputs_dict = net(inputs)
            outputs_dict.label_concepless_tensors(score_treshold = 6.1977e-05)
            pred_meddra_code = CV_test.meddra_codes[outputs_dict['output'].argmax()]


        model_answers.append(pred_meddra_code)
        real_answers.append(data['label_codes'])

f1_score(real_answers, model_answers, average='micro')

100%|██████████| 3999/3999 [00:41<00:00, 96.32batch/s] 


0.695173793448362

In [65]:
outputs_dict

{'output': tensor([[3.9106e-05, 4.1192e-05, 4.0345e-05,  ..., 4.0723e-05, 3.8689e-05,
         3.7171e-05]], device='cuda:0'), 'max_scores': tensor([8.9186e-05], device='cuda:0')}

Инференс

In [49]:
phrase = 'Хуй соси губой тряси ff'
model_path = 'DeepPavlov/rubert-base-cased'
tokenizer = AutoTokenizer.from_pretrained(model_path)
encoded_input = tokenizer([phrase], padding=True, truncation=True, return_tensors='pt')

with torch.no_grad():
    outputs_dict = net(inputs)
    outputs_dict.label_concepless_tensors(score_treshold = 6.1977e-05)
    pred_meddra_code = CV.meddra_codes[outputs_dict['output'].argmax()]
    
print('phrase: %s'%phrase)
print('model: %s'%CV.meddra_code_to_meddra_term[pred_meddra_code])

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


phrase: Хуй соси губой тряси ff
model: Гнев


<h2>Тест модели с mean_pooling</h2>

In [125]:
from sklearn.metrics import classification_report, f1_score
from tqdm import tqdm


net.eval()

model_answers=[]
real_answers=[]

testloader = torch.utils.data.DataLoader(RDR_test, batch_size=1,
                                          shuffle=False)


with tqdm(testloader, unit="batch") as eval_process:
    for data in eval_process:

        #tepoch.set_description(f"Progress")

        inputs = data['tokenized_phrases']
        #labels = data['one_hot_labels']

        with torch.no_grad():
            pred_meddra_code = CV.meddra_codes[net(inputs).argmax()]


        model_answers.append(pred_meddra_code)
        real_answers.append(data['label_codes'])

f1_score(real_answers, model_answers, average='micro')

100%|██████████| 1749/1749 [00:20<00:00, 83.56batch/s] 


0.8147512864493998

<h2>Тест модели с mean_pooling</h2>

In [15]:
from sklearn.metrics import classification_report, f1_score
from tqdm import tqdm


net.eval()

model_answers=[]
real_answers=[]

testloader = torch.utils.data.DataLoader(RDR_test, batch_size=1,
                                          shuffle=False)


with tqdm(testloader, unit="batch") as eval_process:
    for data in eval_process:

        #tepoch.set_description(f"Progress")

        inputs = data['tokenized_phrases']
        labels = data['one_hot_labels']

        with torch.no_grad():
            pred_meddra_code = CV.meddra_codes[net(inputs).argmax()]


        model_answers.append(pred_meddra_code)
        real_answers.append(data['label_codes'])

f1_score(real_answers, model_answers, average='micro')

100%|██████████| 1750/1750 [00:17<00:00, 98.77batch/s] 


0.7994285714285714

In [19]:
from sklearn.metrics import classification_report, f1_score
from tqdm import tqdm


net.eval()

model_answers=[]
real_answers=[]

testloader = torch.utils.data.DataLoader(RDR_test, batch_size=1,
                                          shuffle=False)


with tqdm(testloader, unit="batch") as eval_process:
    for data in eval_process:

        #tepoch.set_description(f"Progress")

        inputs = data['tokenized_phrases']
        labels = data['one_hot_labels']

        with torch.no_grad():
            pred_meddra_code = CV.meddra_codes[net(inputs).argmax()]


        model_answers.append(pred_meddra_code)
        real_answers.append(data['label_codes'])

f1_score(real_answers, model_answers, average='micro')

100%|██████████| 1750/1750 [00:21<00:00, 81.52batch/s]


0.72

In [32]:
from sklearn.metrics import classification_report, f1_score
from tqdm import tqdm


net.eval()

model_answers=[]
real_answers=[]

testloader = torch.utils.data.DataLoader(RDR_test, batch_size=1,
                                          shuffle=False)


with tqdm(testloader, unit="batch") as eval_process:
    for data in eval_process:

        #tepoch.set_description(f"Progress")

        inputs = data['tokenized_phrases']
        labels = data['one_hot_labels']

        with torch.no_grad():
            pred_meddra_code = CV.meddra_codes[net(inputs).argmax()]


        model_answers.append(pred_meddra_code)
        real_answers.append(data['label_codes'][0])

f1_score(real_answers, model_answers, average='micro')

100%|██████████| 1750/1750 [00:20<00:00, 87.42batch/s]


0.792

<h2> Тест необученной модели (на всякий случай) </h2>

In [25]:
from sklearn.metrics import classification_report, f1_score
from tqdm import tqdm

#net.train()
net.eval()

model_answers=[]
real_answers=[]

testloader = torch.utils.data.DataLoader(RDR_test, batch_size=1,
                                          shuffle=False)

with tqdm(testloader, unit="batch") as eval_process:
    for data in eval_process:

        #tepoch.set_description(f"Progress")

        inputs = data['tokenized_phrases']
        labels = data['one_hot_labels']

        with torch.no_grad():
            pred_meddra_code = CV.meddra_codes[net(inputs).argmax()]


        model_answers.append(pred_meddra_code)
        real_answers.append(data['label_codes'])

f1_score(real_answers, model_answers, average='micro')

100%|██████████| 1750/1750 [00:05<00:00, 294.15batch/s]


0.17142857142857143

<h2>Инференс</h2>

In [28]:
from random import randint

#i = randint(1, len(RDR_test))

net.eval()

#phrase = {k: tensor.unsqueeze(0) for k, tensor in RDR_test[i]['tokenized_phrases'].items()}
concept = RDR_test[i]['label_codes']

with torch.no_grad():
    model_answer = CV.meddra_codes[net(phrase).argmax()]
    


print('phrase: %s'%RDR_test[i]['phrases'])
print('model: %s'%CV.meddra_code_to_meddra_term[model_answer])



print('real: %s'%RDR_test[i]['label_terms'])

phrase: температурой 37,8
model: Пирексия
real: Пирексия


Где ошиблась модель

In [51]:
#net = torch.load('./cadec_SoTa_on_RDR_rubert_base_2_epoch.pt')['param_groups']

net.eval()

model_wrong_answers = []
gold_truth_answers = []
phrases = []

testloader = torch.utils.data.DataLoader(RDR_test, batch_size=1,
                                          shuffle=False)

with tqdm(testloader, unit="batch") as eval_process:
    for data in eval_process:

        #tepoch.set_description(f"Progress")

        inputs = data['tokenized_phrases']
        labels = data['one_hot_labels']

        with torch.no_grad():
            pred_meddra_code = CV.meddra_codes[net(inputs).argmax()]
        
        if str(pred_meddra_code)!=str(data['label_codes'][0]):
            model_wrong_answers.append(pred_meddra_code)
            gold_truth_answers.append(data['label_codes'][0])
            phrases.append(data['phrases'][0])
        #model_wrong_answers
        #model_answers.append(pred_meddra_code)
        #real_answers.append(data['label_codes'])
        
import pandas as pd

df = pd.DataFrame(columns=['phrase', 'system output', 'gold markup'])

for phrase, m_a, g_a in zip(phrases, model_wrong_answers, gold_truth_answers):
    new_row = pd.DataFrame({'phrase': [phrase], 'system output': [m_a], 'gold markup': [g_a]})
    df = pd.concat([df, new_row], join='inner', ignore_index=True)
    #gold_truth_answers.append(g_a)
    
df

100%|██████████| 1750/1750 [00:19<00:00, 88.99batch/s]


,phrase,system output,gold markup
0,раздрожалась,10025482,10022998
1,чувство тревоги,10033670,10002855
2,угнетенном состоянии,10025482,10040007
3,Начала плохо спать,10022437,10062519
4,ухудшилось эмоциональное состояние,10061284,10014551
...,...,...,...
359,проблема с кожей на лице,10015150,10000496
360,шаткая нервная система,10029216,10003549
361,острой респираторной вирусной инфекции,10074831,10062352
362,с гнойно-слизистым секретом,10023848,10039083


In [59]:
df.to_csv('rubert_base_RDR_wrong.csv', index=False)

In [74]:
CV.meddra_code_to_meddra_term['CONCEPT_LESS'] = 'CONCEPT_LESS'

In [89]:
import pandas as pd

df = pd.read_csv('elastic_wrong_predictions.csv')
df['system concept'] = df.apply(lambda x: CV.meddra_code_to_meddra_term[str(x['system output'])], axis=1)#], join='outer', ignore_index=True)
df['gold concept'] = df.apply(lambda x: CV.meddra_code_to_meddra_term[str(x['gold markup'])], axis=1)
df

,phrase,system output,gold markup,system concept,gold concept
0,профилактики,10036898,10036898,Профилактика,Профилактика
1,для профилактики,10036898,10036898,Профилактика,Профилактика
2,профилактики,10036898,10036898,Профилактика,Профилактика
3,профилактики,10036898,10036898,Профилактика,Профилактика
4,стрессы,10042209,10042209,Стресс,Стресс
...,...,...,...,...,...
1745,тревожности,CONCEPT_LESS,10033670,CONCEPT_LESS,Паническая реакция
1746,ослабленного иммунитета,CONCEPT_LESS,10021425,CONCEPT_LESS,Нарушение со стороны иммунной системы
1747,чихал,CONCEPT_LESS,10041232,CONCEPT_LESS,Чихание
1748,кашлял,CONCEPT_LESS,10011224,CONCEPT_LESS,Кашель


In [95]:
df_wrong = df[df['system output'].apply(lambda x: int(x) if x!='CONCEPT_LESS' else 'CONCEPT_LESS')!=df['gold markup']]

In [94]:
df_wrong[df_wrong['system output']!='CONCEPT_LESS']

,phrase,system output,gold markup,system concept,gold concept


In [96]:
df_wrong.to_csv('elastic_RDR_wrong.csv', index=False)

In [80]:
df['system output']

0       True
1       True
2       True
3       True
4       True
        ... 
1745    True
1746    True
1747    True
1748    True
1749    True
Length: 1750, dtype: bool

In [82]:
df['system output']

0           10036898
1           10036898
2           10036898
3           10036898
4           10042209
            ...     
1745    CONCEPT_LESS
1746    CONCEPT_LESS
1747    CONCEPT_LESS
1748    CONCEPT_LESS
1749        10005911
Name: system output, Length: 1750, dtype: object

In [81]:
df['gold markup']

0       10036898
1       10036898
2       10036898
3       10036898
4       10042209
          ...   
1745    10033670
1746    10021425
1747    10041232
1748    10011224
1749    10005911
Name: gold markup, Length: 1750, dtype: int64

In [71]:
df.to_csv('rubert_base_wrong_preds_RDR.csv', index=False)

In [44]:
df.drop('Unnamed: 0', axis=1)

,system output,gold markup
0,10025482,10022998
1,10033670,10002855
2,10025482,10040007
3,10022437,10062519
4,10061284,10014551
...,...,...
359,10015150,10000496
360,10029216,10003549
361,10074831,10062352
362,10023848,10039083


In [33]:
df = pd.DataFrame(columns=['system output', 'gold markup'])

for m_a, g_a in zip(model_wrong_answers, gold_truth_answers):
    new_row = pd.DataFrame({'system output': [m_a], 'gold markup': [g_a]})
    df = pd.concat([df, new_row], join='inner', ignore_index=True)
    #gold_truth_answers.append(g_a)
    
df

,system output,gold markup
0,10036898,10036898
1,10036898,10036898
2,10036898,10036898
3,10036898,10036898
4,10042209,10042209
...,...,...
1745,10033670,10033670
1746,10021425,10021425
1747,10041232,10041232
1748,10011224,10011224


In [19]:
net.keys()

dict_keys(['state', 'param_groups'])

<h2>Сохранение и загрузка модели</h2>

In [27]:
torch.save(net, './cadec_SoTa_on_RDR_rubert_base_2_epoch.pt')
torch.save(optimizer.state_dict(), './cadec_SoTa_on_RDR_rubert_base_2_epoch_opt.pt')

In [30]:
the_model = torch.load('./cadec_SoTa_on_RDR_rubert_right_exp.pt')

Покажем, что это ТА ЖЕ модель

In [32]:
the_model.eval()

model_answers=[]
real_answers=[]

testloader = torch.utils.data.DataLoader(RDR_test, batch_size=1,
                                          shuffle=False)


with tqdm(testloader, unit="batch") as eval_process:
    for data in eval_process:


        inputs = data['tokenized_phrases']
        labels = data['one_hot_labels']

        with torch.no_grad():
            pred_meddra_code = CV.meddra_codes[the_model(inputs).argmax()]


        model_answers.append(pred_meddra_code)
        real_answers.append(data['label_codes'])

f1_score(real_answers, model_answers, average='micro')

100%|██████████| 1750/1750 [00:06<00:00, 259.93batch/s]


0.5891428571428572